# Initial evaluation with mini set

In [8]:
matching_algos = ['greedy','hungarian']
yaw_corr = ['True','False']
sim_metrics = ['dist2', 'dist5', 'dist10']
proc_models = ['ack','cvcy','cvcy_obj']
mgmt_methods = ['count','conf_padd_05','conf_padd_10','conf_padd_15','conf_padd_20','conf_padd_25','conf_padd_30','conf_padd_40','conf_padd_50','conf_mult_05','conf_mult_10','conf_mult_15','conf_mult_20','conf_mult_25','conf_mult_30','conf_mult_40','conf_mult_50']

obj_classes= {
    'bicycle': (3,2),
    'bus': (1,2),
    'car': (1,2),
    'motorcycle': (3,2),
    'pedestrian': (1,2),
    'trailer': (3,2),
    'truck': (3,2)
}

det_classes={
    'barrier': ('True','\'\''),
    'bicycle': ('False','bicycle'),
    'bus': ('False','bus'),
    'car': ('False','car'),
    'construction_vehicle': ('True','\'\''),
    'motorcycle': ('False','motorcycle'),
    'pedestrian': ('False','pedestrian'),
    'traffic_cone': ('True','\'\''),
    'trailer': ('False','trailer'),
    'truck': ('False','truck')
}

In [9]:
cfg_names = []

for algo in matching_algos:
    for yc in yaw_corr:
        for metric in sim_metrics:
            for model in proc_models:
                for mgmt in mgmt_methods:

                    # header / tracker params
                    cfg_text = ''
                    cfg_text += 'tbd_tracker_node:\n'
                    cfg_text += '  ros__parameters:\n'
                    cfg_text += '    tracker:\n'
                    cfg_text += '      frame_id: \'map\'\n'
                    cfg_text += '      mismatch_penalty: 100.\n'
                    cfg_text += '      assignment_algo: \'' + algo + '\'\n' 
                    cfg_text += '      yaw_corr: \'' + yc + '\'\n'
                    cfg_text += '      publishers:\n'
                    cfg_text += '        names: [\'track_pub\']\n'
                    cfg_text += '        track_pub:\n'
                    cfg_text += '          pub_topic: \'tracks\'\n'
                    cfg_text += '          msg_type: \'Tracks3D\'\n'
                    cfg_text += '          function: \'publish_tracks\'\n'
                    cfg_text += '          queue_size: 10\n\n'
                
                    # Object properties
                    cfg_text += '    object_properties:\n'
                    cfg_text += '      object_classes: [\'bicycle\',\'bus\',\'car\',\'motorcycle\',\'pedestrian\',\'trailer\',\'truck\']\n\n'
        
                    for obj in obj_classes.keys():
                        cfg_text += '      ' + obj + ':\n'
                        cfg_text += '        model_type: \'' + model + '\'\n'
                        if model=='ack':
                            cfg_text += '        yaw_proc_var: [1.]\n'
                            cfg_text += '        size_proc_var: [1., 1., 1.]\n'
                            cfg_text += '        vel_proc_var: [.1]\n'
                            cfg_text += '        curv_proc_var: [.1]\n'
                            
                        elif 'cvcy' in model:
                            cfg_text += '        yaw_proc_var: [1.]\n'
                            cfg_text += '        size_proc_var: [1., 1., 1.]\n'
                            cfg_text += '        vel_proc_var: [1., 1., 1.]\n'
                            
                        if mgmt=='count':
                            cfg_text += '        create_method: \'count\'\n'
                            cfg_text += '        delete_method: \'count\'\n'
                            cfg_text += '        n_create_min: ' + str((obj_classes[obj][0])) + '\n'
                            cfg_text += '        n_delete_max: ' + str((obj_classes[obj][1]))+ '\n'
                        elif 'conf' in mgmt:
                            cfg_text += '        create_method: \'conf\'\n'
                            cfg_text += '        delete_method: \'conf\'\n'
                            cfg_text += '        active_thresh: .75\n'
                            cfg_text += '        delete_thresh: .15\n'
                            cfg_text += '        detect_thresh: .15\n'
                            if 'padd' in mgmt:
                                cfg_text += '        score_update_function: \'parallel_add\'\n'
                            elif 'mult' in mgmt:
                                cfg_text += '        score_update_function: \'multiply\'\n'

                            if '05' in mgmt:
                                cfg_text += '        score_decay: .05\n'
                            elif '10' in mgmt:
                                cfg_text += '        score_decay: .10\n'
                            elif '15' in mgmt:
                                cfg_text += '        score_decay: .15\n'
                            elif '20' in mgmt:
                                cfg_text += '        score_decay: .20\n'
                            elif '25' in mgmt:
                                cfg_text += '        score_decay: .25\n'
                            elif '30' in mgmt:
                                cfg_text += '        score_decay: .30\n'
                            elif '40' in mgmt:
                                cfg_text += '        score_decay: .40\n'                            
                            elif '50' in mgmt:
                                cfg_text += '        score_decay: .50\n'   

                        if 'dist' in metric:
                            cfg_text += '        sim_metric: \'dist_3d\'\n'
                            if '2' in metric:
                                cfg_text += '        match_thresh: 2.0\n\n'
                            elif '5' in metric:
                                cfg_text += '        match_thresh: 5.0\n\n'
                            elif '10' in metric:
                                cfg_text += '        match_thresh: 10.0\n\n'
                    
                    # Add detectors
                    cfg_text += '    detectors:\n'
                    cfg_text += '      detector_names: [\'megvii\']\n'
                    cfg_text += '      megvii:\n'
                    cfg_text += '        topic: \'converted_detections\'\n'
                    cfg_text += '        msg_type: \'Detections3D\'\n'
                    cfg_text += '        detector_type: \'pos_bbox_3d\'\n'
                    cfg_text += '        detection_classes: [\'barrier\',\'bicycle\',\'bus\',\'car\',\'construction_vehicle\',\'motorcycle\',\'pedestrian\',\'traffic_cone\',\'trailer\',\'truck\']\n'
                    cfg_text += '        detection_properties:\n'
                    for det in det_classes.keys():
                        cfg_text += '          ' + det + ':\n'
                        cfg_text += '            pos_obs_var: [1., 1., 1.]\n'
                        cfg_text += '            yaw_obs_var: [1.]\n'
                        cfg_text += '            size_obs_var: [1., 1., 1.]\n'
                        cfg_text += '            ignore: ' + str((det_classes[det][0])) + '\n'
                        cfg_text += '            object_class: ' + str((det_classes[det][1])) + '\n'
                        
                    
                    
                    # output config
                    filename = "config/%s_%s_%s_%s_%s.yaml" % (algo, metric, model, mgmt, 'yc' if yc=='True' else 'noyc')
                    f = open(filename, 'w')
                    f.write(cfg_text)
                    f.close()
                    cfg_names.append(filename)


In [10]:
print(len(cfg_names))
cfg_names

612


['config/greedy_dist2_ack_count_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_05_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_10_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_15_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_20_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_25_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_30_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_40_yc.yaml',
 'config/greedy_dist2_ack_conf_padd_50_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_05_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_10_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_15_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_20_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_25_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_30_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_40_yc.yaml',
 'config/greedy_dist2_ack_conf_mult_50_yc.yaml',
 'config/greedy_dist2_cvcy_count_yc.yaml',
 'config/greedy_dist2_cvcy_conf_padd_05_yc.yaml',
 'config/greedy_dist2_cvcy_conf_padd_10_yc.yaml',
 'config/greedy_dist2_cvcy_conf

# Second run on val set with yaw corr, matching, and process model set

In [11]:
matching_algos = ['greedy']
yaw_corr = ['False']
sim_metrics = ['dist2', 'dist5', 'dist10']
mgmt_methods = ['count','conf_padd_05','conf_padd_10','conf_padd_15','conf_padd_20','conf_padd_25','conf_padd_30','conf_padd_40','conf_padd_50','conf_mult_05','conf_mult_10','conf_mult_15','conf_mult_20','conf_mult_25','conf_mult_30','conf_mult_40','conf_mult_50']

obj_classes= {
    'bicycle': (3,2,'cvcy'),
    'bus': (1,2,'ack'),
    'car': (1,2,'cvcy'),
    'motorcycle': (3,2,'ack'),
    'pedestrian': (1,2,'cvcy'),
    'trailer': (3,2,'ack'),
    'truck': (3,2,'cvcy')
}

det_classes={
    'barrier': ('True','\'\''),
    'bicycle': ('False','bicycle'),
    'bus': ('False','bus'),
    'car': ('False','car'),
    'construction_vehicle': ('True','\'\''),
    'motorcycle': ('False','motorcycle'),
    'pedestrian': ('False','pedestrian'),
    'traffic_cone': ('True','\'\''),
    'trailer': ('False','trailer'),
    'truck': ('False','truck')
}

In [12]:
cfg_names = []

for algo in matching_algos:
    for yc in yaw_corr:
        for metric in sim_metrics:
            for mgmt in mgmt_methods:

                # header / tracker params
                cfg_text = ''
                cfg_text += 'tbd_tracker_node:\n'
                cfg_text += '  ros__parameters:\n'
                cfg_text += '    tracker:\n'
                cfg_text += '      frame_id: \'map\'\n'
                cfg_text += '      mismatch_penalty: 100.\n'
                cfg_text += '      assignment_algo: \'' + algo + '\'\n' 
                cfg_text += '      yaw_corr: \'' + yc + '\'\n'
                cfg_text += '      publishers:\n'
                cfg_text += '        names: [\'track_pub\']\n'
                cfg_text += '        track_pub:\n'
                cfg_text += '          pub_topic: \'tracks\'\n'
                cfg_text += '          msg_type: \'Tracks3D\'\n'
                cfg_text += '          function: \'publish_tracks\'\n'
                cfg_text += '          queue_size: 10\n\n'
            
                # Object properties
                cfg_text += '    object_properties:\n'
                cfg_text += '      object_classes: [\'bicycle\',\'bus\',\'car\',\'motorcycle\',\'pedestrian\',\'trailer\',\'truck\']\n\n'
    
                for obj in obj_classes.keys():
                    model = str((obj_classes[obj][2]))
                    
                    cfg_text += '      ' + obj + ':\n'
                    cfg_text += '        model_type: \'' + model + '\'\n'
                    if model=='ack':
                        cfg_text += '        yaw_proc_var: [1.]\n'
                        cfg_text += '        size_proc_var: [1., 1., 1.]\n'
                        cfg_text += '        vel_proc_var: [.1]\n'
                        cfg_text += '        curv_proc_var: [.1]\n'
                        
                    elif 'cvcy' in model:
                        cfg_text += '        yaw_proc_var: [1.]\n'
                        cfg_text += '        size_proc_var: [1., 1., 1.]\n'
                        cfg_text += '        vel_proc_var: [1., 1., 1.]\n'
                        
                    if mgmt=='count':
                        cfg_text += '        create_method: \'count\'\n'
                        cfg_text += '        delete_method: \'count\'\n'
                        cfg_text += '        n_create_min: ' + str((obj_classes[obj][0])) + '\n'
                        cfg_text += '        n_delete_max: ' + str((obj_classes[obj][1]))+ '\n'
                    elif 'conf' in mgmt:
                        cfg_text += '        create_method: \'conf\'\n'
                        cfg_text += '        delete_method: \'conf\'\n'
                        cfg_text += '        active_thresh: .75\n'
                        cfg_text += '        delete_thresh: .15\n'
                        cfg_text += '        detect_thresh: .15\n'
                        if 'padd' in mgmt:
                            cfg_text += '        score_update_function: \'parallel_add\'\n'
                        elif 'mult' in mgmt:
                            cfg_text += '        score_update_function: \'multiply\'\n'

                        if '05' in mgmt:
                            cfg_text += '        score_decay: .05\n'
                        elif '10' in mgmt:
                            cfg_text += '        score_decay: .10\n'
                        elif '15' in mgmt:
                            cfg_text += '        score_decay: .15\n'
                        elif '20' in mgmt:
                            cfg_text += '        score_decay: .20\n'
                        elif '25' in mgmt:
                            cfg_text += '        score_decay: .25\n'
                        elif '30' in mgmt:
                            cfg_text += '        score_decay: .30\n'
                        elif '40' in mgmt:
                            cfg_text += '        score_decay: .40\n'                            
                        elif '50' in mgmt:
                            cfg_text += '        score_decay: .50\n'   

                    if 'dist' in metric:
                        cfg_text += '        sim_metric: \'dist_3d\'\n'
                        if '2' in metric:
                            cfg_text += '        match_thresh: 2.0\n\n'
                        elif '5' in metric:
                            cfg_text += '        match_thresh: 5.0\n\n'
                        elif '10' in metric:
                            cfg_text += '        match_thresh: 10.0\n\n'
                
                # Add detectors
                cfg_text += '    detectors:\n'
                cfg_text += '      detector_names: [\'megvii\']\n'
                cfg_text += '      megvii:\n'
                cfg_text += '        topic: \'converted_detections\'\n'
                cfg_text += '        msg_type: \'Detections3D\'\n'
                cfg_text += '        detector_type: \'pos_bbox_3d\'\n'
                cfg_text += '        detection_classes: [\'barrier\',\'bicycle\',\'bus\',\'car\',\'construction_vehicle\',\'motorcycle\',\'pedestrian\',\'traffic_cone\',\'trailer\',\'truck\']\n'
                cfg_text += '        detection_properties:\n'
                for det in det_classes.keys():
                    cfg_text += '          ' + det + ':\n'
                    cfg_text += '            pos_obs_var: [1., 1., 1.]\n'
                    cfg_text += '            yaw_obs_var: [1.]\n'
                    cfg_text += '            size_obs_var: [1., 1., 1.]\n'
                    cfg_text += '            ignore: ' + str((det_classes[det][0])) + '\n'
                    cfg_text += '            object_class: ' + str((det_classes[det][1])) + '\n'
                    
                
                
                # output config
                filename = "config/%s_%s_%s_%s_%s.yaml" % (algo, metric, 'bestmodel', mgmt, 'yc' if yc=='True' else 'noyc')
                f = open(filename, 'w')
                f.write(cfg_text)
                f.close()
                cfg_names.append(filename)


In [13]:
print(len(cfg_names))
cfg_names

51


['config/greedy_dist2_bestmodel_count_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_05_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_10_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_15_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_20_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_25_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_30_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_40_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_padd_50_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_05_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_10_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_15_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_20_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_25_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_30_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_40_noyc.yaml',
 'config/greedy_dist2_bestmodel_conf_mult_50_noyc.yaml',
 'config/greedy_dist5_bestmodel_count_